# CSIRO Feeder Screening

### Goal:
- Scan CSIRO feeders (https://data.csiro.au/collection/csiro:62996)
- Identify small, interpretable radial networks
- Select a feeder suitable for baseline EV experiments


## Screening- Four-wire feeders

In [1]:
using PowerModelsDistribution
using DataFrames

BASE_DIR = joinpath("..", "data", "Four-wire")

results = DataFrame(
    network = String[],
    feeder = String[],
    buses = Int[],
    loads = Int[],
    branches = Int[]
)

for net in sort(readdir(BASE_DIR))
    net_path = joinpath(BASE_DIR, net)
    isdir(net_path) || continue

    for feeder in sort(readdir(net_path))
        feeder_path = joinpath(net_path, feeder)
        master = joinpath(feeder_path, "Master.dss")

        isfile(master) || continue

        try
            pm = PowerModelsDistribution.parse_file(master)

            n_buses = length(pm["bus"])
            n_loads = haskey(pm, "load") ? length(pm["load"]) : 0
            n_lines = haskey(pm, "line") ? length(pm["line"]) : 0
            n_transformers = haskey(pm, "transformer") ? length(pm["transformer"]) : 0

            push!(results, (
                net,
                feeder,
                n_buses,
                n_loads,
                n_lines + n_transformers
            ))
        catch e
            @warn "Failed to load $net / $feeder"
        end
    end
end

results


[ PowerModelsDistribution | Info ] : Circuit has been reset with the 'clear' on line 18 in 'Master.dss'
[ PowerModelsDistribution | Info ] : Redirecting to 'linecode.txt' on line 23 in 'Master.dss'
[ PowerModelsDistribution | Info ] : Redirecting to 'lines.txt' on line 24 in 'LineCode.txt'
[ PowerModelsDistribution | Info ] : Redirecting to 'loads.txt' on line 25 in 'Lines.txt'
[ PowerModelsDistribution | Info ] : Command 'solve' on line 30 in 'Master.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : Command 'closedi' on line 31 in 'Master.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : basemva=100 is the default value, you may want to adjust sbase_default for better convergence
┌ PowerModelsDistribution | Warning ] : basefreq=60.0 on line.line865 does not match circuit basefreq=50.0
└ @ PowerModelsDistribution ~/.julia/packages/PowerModelsDistribution/EOTxp/src/data_model/transformations/dss2eng.jl:422
┌ PowerModelsDistribution | Warning ] : b

Row,network,feeder,buses,loads,branches
,String,String,Int64,Int64,Int64
1,Network_14,Feeder_1,2349,72,2349
2,Network_14,Feeder_2,108,28,108
3,Network_14,Feeder_3,156,60,156
4,Network_14,Feeder_4,1795,68,1795
5,Network_14,Feeder_5,145,45,145
6,Network_14,Feeder_6,176,44,176
7,Network_24,Feeder_1,1076,56,1076
8,Network_24,Feeder_2,1694,79,1694
9,Network_8,Feeder_1,2068,52,2068


In [2]:
sort(results, :buses)

Row,network,feeder,buses,loads,branches
,String,String,Int64,Int64,Int64
1,network_23,Feeder_3,59,2,59
2,network_20,Feeder_5,65,23,65
3,network_20,Feeder_4,71,29,71
4,network_20,Feeder_1,78,25,78
5,network_9,Feeder_3,107,20,107
6,Network_14,Feeder_2,108,28,108
7,network_13,Feeder_3,108,2,108
8,network_9,Feeder_2,112,21,112
9,network_5,Feeder_1,131,4,131


In [3]:
small = filter(r -> r.buses ≤ 150 && r.loads ≥ 10, results)
sort(small, :buses)

Row,network,feeder,buses,loads,branches
,String,String,Int64,Int64,Int64
1,network_20,Feeder_5,65,23,65
2,network_20,Feeder_4,71,29,71
3,network_20,Feeder_1,78,25,78
4,network_9,Feeder_3,107,20,107
5,Network_14,Feeder_2,108,28,108
6,network_9,Feeder_2,112,21,112
7,network_9,Feeder_4,140,21,140
8,Network_14,Feeder_5,145,45,145


### Selected feeder:
- Network: network_20
- Feeder: Feeder_5
- Buses: 65
- Loads: 23
>Reason: small, radial, suitable for intuition-building


## Screening – Three-wire Kron-reduced feeders



Due to current PowerModelsDistribution limitations in solving
grounded four-wire LV feeders, screening is repeated using the
three-wire Kron-reduced representation of the CSIRO dataset.

This representation preserves feeder topology and loading,
while enabling stable power flow and OPF simulations.

In [1]:
using PowerModelsDistribution
using DataFrames

BASE_DIR_3W = joinpath("..", "data", "Three-wire-Kron-reduced")

results_3w = DataFrame(
    network = String[],
    feeder = String[],
    buses = Int[],
    loads = Int[],
    branches = Int[]
)

for net in sort(readdir(BASE_DIR_3W))
    net_path = joinpath(BASE_DIR_3W, net)
    isdir(net_path) || continue

    for feeder in sort(readdir(net_path))
        feeder_path = joinpath(net_path, feeder)
        master = joinpath(feeder_path, "Master.dss")

        isfile(master) || continue

        try
            pm = PowerModelsDistribution.parse_file(master)

            n_buses = length(pm["bus"])
            n_loads = haskey(pm, "load") ? length(pm["load"]) : 0
            n_lines = haskey(pm, "line") ? length(pm["line"]) : 0
            n_transformers = haskey(pm, "transformer") ? length(pm["transformer"]) : 0

            push!(results_3w, (
                net,
                feeder,
                n_buses,
                n_loads,
                n_lines + n_transformers
            ))
        catch
            @warn "Failed to load $net / $feeder"
        end
    end
end

results_3w


[ PowerModelsDistribution | Info ] : Circuit has been reset with the 'clear' on line 18 in 'Master.dss'
[ PowerModelsDistribution | Info ] : Redirecting to 'linecode.txt' on line 22 in 'Master.dss'
[ PowerModelsDistribution | Info ] : Redirecting to 'lines.txt' on line 23 in 'LineCode.txt'
[ PowerModelsDistribution | Info ] : Redirecting to 'loads.txt' on line 24 in 'Lines.txt'
[ PowerModelsDistribution | Info ] : Command 'solve' on line 29 in 'Master.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : Command 'closedi' on line 30 in 'Master.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : basemva=100 is the default value, you may want to adjust sbase_default for better convergence
┌ PowerModelsDistribution | Warning ] : basefreq=60.0 on line.line865 does not match circuit basefreq=50.0
└ @ PowerModelsDistribution ~/.julia/packages/PowerModelsDistribution/EOTxp/src/data_model/transformations/dss2eng.jl:422
┌ PowerModelsDistribution | Warning ] : b

Row,network,feeder,buses,loads,branches
,String,String,Int64,Int64,Int64
1,Network_14,Feeder_1,2349,72,2348
2,Network_14,Feeder_2,108,28,107
3,Network_14,Feeder_3,156,60,155
4,Network_14,Feeder_4,1795,68,1794
5,Network_14,Feeder_5,145,45,144
6,Network_14,Feeder_6,176,44,175
7,Network_24,Feeder_1,1076,56,1075
8,Network_24,Feeder_2,1694,79,1693
9,Network_8,Feeder_1,2068,52,2067


In [2]:
sort(results_3w, :buses)

Row,network,feeder,buses,loads,branches
,String,String,Int64,Int64,Int64
1,network_23,Feeder_3,59,2,58
2,network_20,Feeder_5,65,23,64
3,network_20,Feeder_4,71,29,70
4,network_20,Feeder_1,78,25,77
5,network_9,Feeder_3,107,20,106
6,Network_14,Feeder_2,108,28,107
7,network_13,Feeder_3,108,2,107
8,network_9,Feeder_2,112,21,111
9,network_5,Feeder_1,131,4,130


In [3]:
small_3W = filter(r -> r.buses ≤ 150 && r.loads ≥ 10, results_3w)
sort(small_3W, :buses)

Row,network,feeder,buses,loads,branches
,String,String,Int64,Int64,Int64
1,network_20,Feeder_5,65,23,64
2,network_20,Feeder_4,71,29,70
3,network_20,Feeder_1,78,25,77
4,network_9,Feeder_3,107,20,106
5,Network_14,Feeder_2,108,28,107
6,network_9,Feeder_2,112,21,111
7,network_9,Feeder_4,140,21,139
8,Network_14,Feeder_5,145,45,144


## Screening – Three-wire phase-to-neutral feeders




Due to current limitations in PowerModelsDistribution when solving
grounded four-wire LV feeders, network screening is performed using
the **three-wire phase-to-neutral (Kron-reduced) representation** of
the CSIRO dataset.

This representation:
- preserves the original feeder topology and load placement,
- embeds the neutral conductor’s effect into the phase impedances,
- enables stable and repeatable power flow and OPF simulations.

All feeders selected in this step are therefore suitable for baseline
and scenario analysis.

In [6]:
using PowerModelsDistribution
using DataFrames

BASE_DIR_3W_PH = joinpath("..", "data", "Three-wire-phase-to-neutral")

results_3w_ph = DataFrame(
    network = String[],
    feeder = String[],
    buses = Int[],
    loads = Int[],
    branches = Int[]
)

for net in sort(readdir(BASE_DIR_3W_PH))
    net_path = joinpath(BASE_DIR_3W_PH, net)
    isdir(net_path) || continue

    for feeder in sort(readdir(net_path))
        feeder_path = joinpath(net_path, feeder)
        master = joinpath(feeder_path, "Master.dss")

        isfile(master) || continue

        try
            pm_3w_ph = PowerModelsDistribution.parse_file(master)

            n_buses = length(pm_3w_ph["bus"])
            n_loads = haskey(pm_3w_ph, "load") ? length(pm_3w_ph["load"]) : 0
            n_lines = haskey(pm_3w_ph, "line") ? length(pm_3w_ph["line"]) : 0
            n_transformers = haskey(pm_3w_ph, "transformer") ? length(pm_3w_ph["transformer"]) : 0

            push!(results_3w_ph, (
                net,
                feeder,
                n_buses,
                n_loads,
                n_lines + n_transformers
            ))
        catch
            @warn "Failed to load $net / $feeder"
        end
    end
end

results_3w_ph


[ PowerModelsDistribution | Info ] : Circuit has been reset with the 'clear' on line 18 in 'Master.dss'
[ PowerModelsDistribution | Info ] : Redirecting to 'linecode.txt' on line 22 in 'Master.dss'
[ PowerModelsDistribution | Info ] : Redirecting to 'lines.txt' on line 23 in 'LineCode.txt'
[ PowerModelsDistribution | Info ] : Redirecting to 'loads.txt' on line 24 in 'Lines.txt'
[ PowerModelsDistribution | Info ] : Command 'solve' on line 29 in 'Master.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : Command 'closedi' on line 30 in 'Master.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : basemva=100 is the default value, you may want to adjust sbase_default for better convergence
┌ PowerModelsDistribution | Warning ] : basefreq=60.0 on line.line865 does not match circuit basefreq=50.0
└ @ PowerModelsDistribution ~/.julia/packages/PowerModelsDistribution/EOTxp/src/data_model/transformations/dss2eng.jl:422
┌ PowerModelsDistribution | Warning ] : b

Row,network,feeder,buses,loads,branches
,String,String,Int64,Int64,Int64
1,Network_14,Feeder_1,2349,72,2348
2,Network_14,Feeder_2,108,28,107
3,Network_14,Feeder_3,156,60,155
4,Network_14,Feeder_4,1795,68,1794
5,Network_14,Feeder_5,145,45,144
6,Network_14,Feeder_6,176,44,175
7,Network_24,Feeder_1,1076,56,1075
8,Network_24,Feeder_2,1694,79,1693
9,Network_8,Feeder_1,2068,52,2067


In [7]:
sort(results_3w_ph, :buses)

Row,network,feeder,buses,loads,branches
,String,String,Int64,Int64,Int64
1,network_23,Feeder_3,59,2,58
2,network_20,Feeder_5,65,23,64
3,network_20,Feeder_4,71,29,70
4,network_20,Feeder_1,78,25,77
5,network_9,Feeder_3,107,20,106
6,Network_14,Feeder_2,108,28,107
7,network_13,Feeder_3,108,2,107
8,network_9,Feeder_2,112,21,111
9,network_5,Feeder_1,131,4,130


In [8]:
small_3w_ph = filter(r -> r.buses ≤ 150 && r.loads ≥ 10, results_3w_ph)
sort(small_3w_ph, :buses)

Row,network,feeder,buses,loads,branches
,String,String,Int64,Int64,Int64
1,network_20,Feeder_5,65,23,64
2,network_20,Feeder_4,71,29,70
3,network_20,Feeder_1,78,25,77
4,network_9,Feeder_3,107,20,106
5,Network_14,Feeder_2,108,28,107
6,network_9,Feeder_2,112,21,111
7,network_9,Feeder_4,140,21,139
8,Network_14,Feeder_5,145,45,144


### Screening results – candidate three-wire feeders

The three-wire phase-to-neutral CSIRO feeders were screened to identify
a **manageable but representative LV network** for baseline analysis.

Selection criteria:
- fewer than 150 buses (to keep simulations tractable),
- more than 10 loads (to ensure meaningful voltage and loading behaviour).

This resulted in the following candidate feeders.
